# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [71]:
import pke
import json
import spacy
from src.pdf_parser import Parser
from pathlib import Path
from tqdm.notebook import tqdm
from gensim.models.fasttext import load_facebook_model
from gensim.models.fasttext import FastText
import compress_fasttext
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from dataclasses import dataclass

# Parsing

In [9]:
parser = Parser()

In [11]:
docs = {}

for x in tqdm(Path("../data/raw/ВМ.5503._Технологии баз данных/2019/Рабочие программы дисциплин").rglob("*.*")):
    if x.suffix in [".docx", '.pdf']:
        docs[x.stem] = parser.parse(x, ext=x.suffix[1:])

In [ ]:
len(docs)

# Keyword extraction

In [90]:
model = spacy.load("../ru2/")

## РПД

In [92]:
rpd_keywords = {}

for x in tqdm(docs):
    extractor = pke.unsupervised.SingleRank()
    extractor.load_document(input=docs[x], spacy_model=model)
    extractor.candidate_selection()
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=30)
    rpd_keywords[x] = keyphrases

100%|██████████| 39/39 [00:30<00:00,  1.27it/s]


In [93]:
len(rpd_keywords)

39

In [95]:
rpd_keywords.keys()

dict_keys(['018999_Введение в искусственный интеллект', '001123_Методы статистической обработки информации', '001134_Машинное обучение', '001111_Современная философия и методология науки', '001117_История развития вычислительной техники и программирования (2)', '001136_Технологии специализированных баз данных для моделирования', '053719_Базы данных для геоинформационных систем_б', 'Ред. Проект РПД_адаптац. дисциплина_правильная_13.12.2018', '053720_Hadoop', '001115_Научно-исследовательская работа (курсовая работа)', '001112_Алгоритмические основы распознавания изображения_2018', '001114_Современные СУБД', '001113_Объектно-ориентированные CASE технологии', '001130_Алгоритмы и технологии анализа данных', '001125_Устойчивость движений дискретных динамических систем', '18_056866_Введение в блокчейн технологии_База', '001127_Информационный поиск в неструктурированных данных', '012978_Grid-технологии', '053715_Промышленные СУБД', '001131_Интеллектуальный анализ данных', '001119_НИПрактика_55

In [96]:
with open("../data/processed/tbd_keywords.json", 'w') as file:
    json.dump(rpd_keywords, file, indent=4, ensure_ascii=False)

## Профстандарты

In [52]:
with open("../data/processed_standards/standards_text.json") as file:
    prof = json.load(file)

In [53]:
prof = {x['standard_name']:x['text'] for x in prof}

In [40]:
prof_keywords = {}

for x in tqdm(prof):
    extractor = pke.unsupervised.SingleRank()
    extractor.load_document(input=prof[x], spacy_model=model)
    extractor.candidate_selection()
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=30)
    prof_keywords[x] = keyphrases

100%|██████████| 1336/1336 [25:43<00:00,  1.16s/it]


In [ ]:
with open("../data/prof_keywords.json", 'w') as file:
    json.dump(prof_keywords, file, indent=4, ensure_ascii=False)

# Similarity based on mean embeddings of keywords

In [12]:
with open('../data/processed/tbd_keywords.json') as f:
    rpd_keywords = json.load(f)
    
with open('../data/processed/prof_keywords.json') as f:
    prof_keywords = json.load(f)

In [15]:
model = compress_fasttext.models.CompressedFastTextKeyedVectors.load('../data/models/pruned_ft_500K.bin')
# model = load_facebook_model('../data/models/pruned_ft_500K.bin')

In [100]:
rpd_embeddings = {}
for name in tqdm(rpd_keywords):
    keywords = [keyword for (keyword, score) in rpd_keywords[name]]
    embedding = model[keywords].mean(axis=0)
    rpd_embeddings[name] = embedding
    
prof_embeddings = {}
for name in tqdm(prof_keywords):
    keywords = [keyword for (keyword, score) in prof_keywords[name]]
    embedding = model[keywords].mean(axis=0)
    prof_embeddings[name] = embedding

100%|██████████| 1336/1336 [00:44<00:00, 30.33it/s]


In [101]:
len(rpd_embeddings), len(prof_embeddings)

(39, 1336)

In [102]:
import pickle

with open('../data/processed/tbd_embeddings.pkl', 'wb') as f:
    pickle.dump(rpd_embeddings, f)
    
with open('../data/processed/prof_embeddings.pkl', 'wb') as f:
    pickle.dump(prof_embeddings, f)

In [103]:
list(zip(*rpd_embeddings.items()))[0]

('018999_Введение в искусственный интеллект',
 '001123_Методы статистической обработки информации',
 '001134_Машинное обучение',
 '001111_Современная философия и методология науки',
 '001117_История развития вычислительной техники и программирования (2)',
 '001136_Технологии специализированных баз данных для моделирования',
 '053719_Базы данных для геоинформационных систем_б',
 'Ред. Проект РПД_адаптац. дисциплина_правильная_13.12.2018',
 '053720_Hadoop',
 '001115_Научно-исследовательская работа (курсовая работа)',
 '001112_Алгоритмические основы распознавания изображения_2018',
 '001114_Современные СУБД',
 '001113_Объектно-ориентированные CASE технологии',
 '001130_Алгоритмы и технологии анализа данных',
 '001125_Устойчивость движений дискретных динамических систем',
 '18_056866_Введение в блокчейн технологии_База',
 '001127_Информационный поиск в неструктурированных данных',
 '012978_Grid-технологии',
 '053715_Промышленные СУБД',
 '001131_Интеллектуальный анализ данных',
 '001119_НИП

# Similarity based on keyword mapping 

In [72]:
@dataclass
class ProfStandard:
    name: str
    rank: int
    rpds: dict


def get_ranked_standards(rpd_keywords, prof_keywords, model, *, threshold=0.9):
    res = []
    for prof_name in tqdm(prof_keywords):
        keywords1 = [keyword for (keyword, score) in prof_keywords[prof_name]]
        prof_embeddings = model[keywords1]
        prof_standard = ProfStandard(name=prof_name, rank=0, rpds={})
        for rpd_name in rpd_keywords:
            keywords2 = [keyword for (keyword, score) in rpd_keywords[rpd_name]]
            rpd_embeddings = model[keywords2]
            similarity_matrix = cosine_similarity(prof_embeddings, rpd_embeddings)
            prof_idxs, rpd_idxs = np.where(similarity_matrix > threshold)
            prof_standard.rank += len(prof_idxs)
            prof_standard.rpds[rpd_name] = set([rpd_keywords[rpd_name][idx][0] for idx in rpd_idxs])
        res.append(prof_standard)
    return res

In [92]:
prof_ranks = get_ranked_standards(rpd_keywords, prof_keywords, model, threshold=0.9)

In [93]:
len(prof_ranks)

1336

In [94]:
sorted_prof_ranks = sorted(prof_ranks, key=lambda item: item.rank, reverse=True)

In [95]:
sorted_prof_ranks[1].rpds

{'018999_Введение в искусственный интеллект': {'характеристики специализированного программного обеспечения'},
 '001123_Методы статистической обработки информации': {'системное программное обеспечение общего назначения'},
 '001134_Машинное обучение': set(),
 '001111_Современная философия и методология науки': set(),
 '001117_История развития вычислительной техники и программирования (2)': {'история языков программирования'},
 '001136_Технологии специализированных баз данных для моделирования': set(),
 '053719_Базы данных для геоинформационных систем_б': set(),
 'Ред. Проект РПД_адаптац. дисциплина_правильная_13.12.2018': set(),
 '053720_Hadoop': set(),
 '001115_Научно-исследовательская работа (курсовая работа)': set(),
 '001112_Алгоритмические основы распознавания изображения_2018': {'характеристики специализированного программного обеспечения'},
 '001114_Современные СУБД': {'овладения критериями выбора системы управления базами данных',
  'стратегии выбора системы управления базами да

In [96]:
for prof in sorted_prof_ranks[:20]:
    print(prof.name, prof.rank)

Руководитель разработки программного обеспечения 338
Системный программист 267
Программист 248
Системный администратор информационно-коммуникационных систем 123
Инженер-конструктор по динамике полета и управлению летательным аппаратом в ракетно-космической промышленности 111
Специалист учебно-тренировочного подразделения атомной станции (технические средства обучения) 99
Специалист по контролю качества информационно-коммуникационных систем 88
Специалист по разработке системы управления полетами ракет-носителей  и космических аппаратов 80
Архитектор программного обеспечения 78
Специалист по наладке и испытаниям технологического оборудования механосборочного производства 75
Специалист в области информационных технологий на атомных станциях (разработка и сопровождение программного обеспечения) 75
Инженер-конструктор аналоговых сложнофункциональных блоков 71
Специалист по проектированию и конструированию систем жизнеобеспечения, терморегулирования, агрегатов пневмогидравлических систем пил